In [63]:
from sklearn.datasets import make_classification
import torch

In [64]:
# Synthetic classification dataset using sklearn
x, y = make_classification(
    n_samples=10,       # Number of samples
    n_features=2,       # Number of features
    n_informative=2,    # Number of informative features
    n_redundant=0,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42     # For reproducibility
)

In [65]:
print(type(x))

<class 'numpy.ndarray'>


In [66]:
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

print(x[0])
print(y[0])

tensor([ 1.0683, -0.9701])
tensor(1)


In [67]:
from torch.utils.data import Dataset, DataLoader


In [68]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return self.features.shape[0]

  def __getitem__(self, index):

    return self.features[index], self.labels[index]

In [69]:
dataset = CustomDataset(x, y)
print(len(dataset)) # len(lst) internally calls lst.__len__().
print(dataset[2])  # lst[2] internally calls lst.__getitem__(2)

10
(tensor([-2.8954,  1.9769]), tensor(0))


In [70]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)
cnt = 1

for batch_features, batch_labels in dataloader:
  
  if cnt : 
    print(batch_features.shape , batch_features[0])
    print(batch_labels.shape , batch_labels[0])
    cnt = 0

  print(batch_features)
  print(batch_labels)

  print("-"*50)

torch.Size([2, 2]) tensor([ 1.0683, -0.9701])
torch.Size([2]) tensor(1)
tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388]])
tensor([1, 0])
--------------------------------------------------
tensor([[-2.8954,  1.9769],
        [-0.7206, -0.9606]])
tensor([0, 0])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [-0.9382, -0.5430]])
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.7273, -1.1858],
        [ 1.7774,  1.5116]])
tensor([1, 1])
--------------------------------------------------
tensor([[ 1.8997,  0.8344],
        [-0.5872, -1.9717]])
tensor([1, 0])
--------------------------------------------------


In [71]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [72]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [73]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)
xtrain,xtest,ytrain,ytest = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [74]:
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.fit_transform(xtest)

encoder = LabelEncoder()
ytrain = encoder.fit_transform(ytrain)
ytest = encoder.fit_transform(ytest)


In [75]:
xtrain_tensor = torch.from_numpy(xtrain).float()
xtest_tensor = torch.from_numpy(xtest).float()
ytrain_tensor = torch.from_numpy(ytrain).float()
ytest_tensor = torch.from_numpy(ytest).float()

In [76]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]


In [77]:
train_dataset = CustomDataset(xtrain_tensor, ytrain_tensor)
test_dataset = CustomDataset(xtest_tensor, ytest_tensor)

print(len(train_dataset) , train_dataset[0][0].shape , train_dataset[0][1].shape)
print(len(test_dataset),test_dataset[0][0].shape , train_dataset[0][1].shape)

455 torch.Size([30]) torch.Size([])
114 torch.Size([30]) torch.Size([])


In [78]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [79]:
loader_iter = iter(train_loader)
first_batch = next(loader_iter)

features, labels = first_batch

print(features.shape, labels.shape)

torch.Size([32, 30]) torch.Size([32])


In [80]:
import torch.nn as nn

class MySimpleNN(nn.Module):

    def __init__(self , num_features):

        super().__init__()

        self.network = nn.Sequential(
            nn.Linear(num_features , 1),
            nn.Sigmoid()
        )
        
    def forward(self , features):
        
        out = self.network(features)
        return out
    
    

In [81]:
from torchinfo import summary

model = MySimpleNN(xtrain_tensor.shape[1])

learning_rate = 0.1
epochs = 25

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_function = nn.BCELoss()

summary(model , input_size = xtrain_tensor.shape)

Layer (type:depth-idx)                   Output Shape              Param #
MySimpleNN                               [455, 1]                  --
├─Sequential: 1-1                        [455, 1]                  --
│    └─Linear: 2-1                       [455, 1]                  31
│    └─Sigmoid: 2-2                      [455, 1]                  --
Total params: 31
Trainable params: 31
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.05
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.06

In [82]:
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    y_pred = model(batch_features)
    
    loss = loss_function(y_pred, batch_labels.unsqueeze(-1))

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.1334846317768097
Epoch: 2, Loss: 0.1871817260980606
Epoch: 3, Loss: 0.029278215020895004
Epoch: 4, Loss: 0.008004295639693737
Epoch: 5, Loss: 0.1672862321138382
Epoch: 6, Loss: 0.1678992360830307
Epoch: 7, Loss: 0.03915240988135338
Epoch: 8, Loss: 0.054214660078287125
Epoch: 9, Loss: 0.0890548974275589
Epoch: 10, Loss: 0.06640886515378952
Epoch: 11, Loss: 0.01623552106320858
Epoch: 12, Loss: 0.039088014513254166
Epoch: 13, Loss: 0.014814257621765137
Epoch: 14, Loss: 0.16895517706871033
Epoch: 15, Loss: 0.03407168760895729
Epoch: 16, Loss: 0.03210907056927681
Epoch: 17, Loss: 0.012332765385508537
Epoch: 18, Loss: 0.13987082242965698
Epoch: 19, Loss: 0.012757226824760437
Epoch: 20, Loss: 0.1726992428302765
Epoch: 21, Loss: 0.12797890603542328
Epoch: 22, Loss: 0.2144133746623993
Epoch: 23, Loss: 0.018826721236109734
Epoch: 24, Loss: 0.7002910375595093
Epoch: 25, Loss: 0.024030262604355812


In [86]:
model.eval() 
accuracy_list = []

with torch.no_grad():

    for batch_features, batch_labels in test_loader:

        y_pred = model(batch_features)
        y_pred = (y_pred > 0.5).float()  

        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()

        accuracy_list.append(batch_accuracy)

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')

# NOTE : t.view(-1) => t.flatten()

Accuracy: 0.9410
